In [276]:
import numpy as np
import pandas as pd
import re
pd.set_option('display.max_rows', 3000)

In [277]:
bishops = pd.read_csv('/users/Orion/NYU/bishop-scraper/european-bishops-wiki-scraper/bishops.csv')

In [278]:
bishops['From'] = bishops['From'].str.extract('([0123456789/]+)', expand=False)
bishops['To'] = bishops['To'].str.extract('([0123456789/]+)', expand=False)

In [279]:
new1 = bishops['From'].str.split('/', expand=True)
bishops['From1'] = new1[0]
bishops['From2'] = new1[1]
new2 = bishops['To'].str.split('/', expand=True)
bishops['To1'] = new2[0]
bishops['To2'] = new2[1]

In [280]:
bishops['To'] = np.where((bishops['From1'].notnull() & bishops['From2'].notnull() & 
                          bishops['To1'].isnull() & bishops['To2'].isnull()), bishops['From2'], bishops['To'])
bishops['From'] = np.where((bishops['To1'].notnull() & bishops['To2'].notnull() & 
                          bishops['From1'].isnull() & bishops['From2'].isnull()), bishops['To1'], bishops['From'])
bishops['From'] = np.where((bishops['From1'].notnull() & bishops['From2'].notnull() & 
                          bishops['To'].notnull()), bishops['From1'], bishops['From'])
bishops['To'] = np.where((bishops['To1'].notnull() & bishops['To2'].notnull() & 
                          bishops['From'].notnull()), bishops['To2'], bishops['To'])
bishops = bishops.drop(columns=['From1', 'From2', 'To1', 'To2'])

In [281]:
bishops['From'] = bishops['From'].astype('str')
bishops['To'] = bishops['To'].astype('str')
bishops['To'] = np.where(bishops['To'] == '0', np.nan, bishops['To'])
bishops['To'] = np.where(bishops['To'].str[0:2] == bishops['From'].str[0:2], np.nan, bishops['To'])
bishops['From'] = np.where(bishops['From'].str[0:2] == bishops['To'].str[0:2], np.nan, bishops['From'])
bishops['To'] = np.where((bishops['To'].str.len() == 2) & (bishops['From'].str.len() == 4), bishops['From'].str[0:2]+bishops['To'], bishops['To'])
bishops['To'] = np.where((bishops['To'].str.len() == 1) & (bishops['From'].str.len() == 4), bishops['From'].str[0:3]+bishops['To'], bishops['To'])
bishops['To'] = np.where((bishops['To'].str.len() == 1) & (bishops['From'].str.len() == 3), bishops['From'].str[0:2]+bishops['To'], bishops['To'])
bishops['From'] = np.where((bishops['From'].str.len() < 3) & (bishops['To'].str.isdigit() == False), np.nan, bishops['From'])

In [283]:
bishops = bishops.replace('nan', np.nan)

In [287]:
bishops_cleaned_years = bishops.to_csv('bishops_cleaned_years.csv')